In [ ]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import Input, LayerNormalization, MultiHeadAttention, Add, Reshape, Flatten, Dense, Dropout, Embedding, Lambda

with h5py.File('dataset.h5', 'r') as f:
    X_train = f['X_train'][:]
    X_val = f['X_val'][:]
    y_train = f['y_train'][:]
    y_val = f['y_val'][:]

print("Loaded preprocessed data:")
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)


Loaded preprocessed data:
X_train shape: (288, 30, 1, 1, 2048)
X_val shape: (72, 30, 1, 1, 2048)
y_train shape: (288,)
y_val shape: (72,)


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

INPUT_SIZE = (299, 299)
FEATURE_DIM = 2048
NUM_FRAMES = 30
TRAIN_REAL = 'extracted_frames/train/real'
TRAIN_FAKE = 'extracted_frames/train/fake'

def transformer_encoder(inputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.1):
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=head_size, dropout=dropout)(inputs, inputs)
    attention = Dropout(dropout)(attention)
    attention_out = Add()([inputs, attention])
    attention_out = LayerNormalization(epsilon=1e-6)(attention_out)

    ffn = Dense(ff_dim, activation="relu")(attention_out)
    ffn = Dropout(dropout)(ffn)
    ffn = Dense(inputs.shape[-1])(ffn)
    output = Add()([attention_out, ffn])
    output = LayerNormalization(epsilon=1e-6)(output)
    
    return output

def build_transformer_model(input_shape=(NUM_FRAMES, 1, 1, FEATURE_DIM)):
    inputs = Input(shape=input_shape)  
    x = Reshape((NUM_FRAMES, FEATURE_DIM))(inputs) 

    positions = tf.range(start=0, limit=NUM_FRAMES, delta=1)
    pos_emb_layer = Embedding(input_dim=NUM_FRAMES, output_dim=FEATURE_DIM)
    pos_embeddings = pos_emb_layer(positions) 
    
    pos_embeddings = Lambda(lambda pe: tf.expand_dims(pe, axis=0))(pos_embeddings) 
    x = Add()([x, pos_embeddings]) 

    for _ in range(2):
        x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=256)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    return model


model = build_transformer_model()
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 30, 1, 1,  │          0 │ -                 │
│ (InputLayer)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 30, 2048)  │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (1, 30, 2048)     │          0 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, 30, 2048)     │  2,099,968 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (1, 30, 2048)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (1, 30, 2048)     │          0 │ add[0][0],        │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (1, 30, 2048)     │      4,096 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (1, 30, 256)      │    524,544 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (1, 30, 256)      │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (1, 30, 2048)     │    526,336 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (1, 30, 2048)     │          0 │ layer_normalizat… │
│                     │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, 30, 2048)     │      4,096 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, 30, 2048)     │  2,099,968 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (1, 30, 2048)     │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (1, 30, 2048)     │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_4[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, 30, 2048)     │      4,096 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (1, 30, 256)      │    524,544 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (1, 30, 256)      │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (1, 30, 2048)     │    526,336 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (1, 30, 2048)     │          0 │ layer_normalizat… │
│                     │                   │            │ dense_3[0][0]   

 Total params: 14,182,657 (54.10 MB)

 Trainable params: 14,182,657 (54.10 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)

model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=8,
    callbacks=[lr_scheduler]
)


Epoch 1/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 25s 278ms/step - accuracy: 0.4889 - loss: 1.9515 - val_accuracy: 0.6389 - val_loss: 0.5724 - learning_rate: 1.0000e-04
Epoch 2/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 240ms/step - accuracy: 0.6926 - loss: 0.6409 - val_accuracy: 0.6806 - val_loss: 0.5627 - learning_rate: 1.0000e-04
Epoch 3/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 238ms/step - accuracy: 0.7660 - loss: 0.5305 - val_accuracy: 0.6944 - val_loss: 0.5754 - learning_rate: 1.0000e-04
Epoch 4/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 229ms/step - accuracy: 0.7123 - loss: 0.5131 - val_accuracy: 0.6806 - val_loss: 0.5314 - learning_rate: 1.0000e-04
Epoch 5/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 222ms/step - accuracy: 0.8272 - loss: 0.4177 - val_accuracy: 0.7083 - val_loss: 0.5286 - learning_rate: 1.0000e-04
Epoch 6/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 9s 249ms/step - accuracy: 0.6870 - loss: 0.4935 - val_accuracy: 0.6806 - val_loss: 0.5422 - learning_rate: 1.0000e-04
Epoch 7/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 10s 265ms/step - accuracy

In [ ]:
y_pred_prob = model.predict(X_val).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

auc_score = roc_auc_score(y_val, y_pred_prob)
print(f"AUC Score: {auc_score:.4f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 451ms/step
AUC Score: 0.8441
